# Welcome to Hypothesis Testing


![Types of Stats](Stats-types.jpg)


![descriptive-and-inferential-statistics](descriptive-and-inferential-statistics.jpeg)



We will use some built-in datasets from the pydataset library to review and explore some concepts. 

1. How distributions help us make inferernces

2. Sample vs. population

3. Asking interesting and relevant questions of your data

4. Ways to answer questions

5. How Hypothesis testing helps us make inferences (In this lesson we will introduce some broad concepts related to hypothesis testing, and in future lessons we will look at examples of 3 types of hypothesis tests in detail.)

6. Key terms in hypothesis testing


In doing all of this, we will also practice transforming datasets using python, understanding the structure of an existing dataset and how it may differ from what you want, how to define an observation vs. a row. 

So, let's get to it...

__________________________________________________

## Data Wrangling

First, we need data. In this lesson, instead of generating random data, we will use data from the pydataset library which contains 756 datasets available for use. 
As you will soon get used to, every data science project starts with prject planning followed by acquiring and preparing data, a.k.a. wrangling the data. One non-negotiable to having prepared data ready to explore is that each row represents an observation. Let's take an example. 

In [6]:
# Using open datasets from pydataset
from pydataset import data

import pandas as pd
import numpy as np

from scipy import stats

import matplotlib.pyplot as plt

In [7]:
# take a look at available datasets 
data()

,dataset_id,title
0,AirPassengers,Monthly Airline Passenger Numbers 1949-1960
1,BJsales,Sales Data with Leading Indicator
2,BOD,Biochemical Oxygen Demand
3,Formaldehyde,Determination of Formaldehyde
4,HairEyeColor,Hair and Eye Color of Statistics Students
...,...,...
752,VerbAgg,Verbal Aggression item responses
753,cake,Breakage Angle of Chocolate Cakes
754,cbpp,Contagious bovine pleuropneumonia
755,grouseticks,Data on red grouse ticks from Elston et al. 2001


We are going to use the **HairEyeColor** dataset. 

In [10]:
# Extract just the row that contains the HairEyeColor dataset using iloc
data().iloc[4]

dataset_id                                 HairEyeColor
title         Hair and Eye Color of Statistics Students
Name: 4, dtype: object

Ok, We are told this dataset contains information about Hair and Eye color of a group of statistics students. 
Let's take a look. 

In [ ]:
# store that data in a dataframe named df
df = data('HairEyeColor')

In [ ]:
# look at the numbers of rows and columns


In [12]:
# look at the first 5 rows


(32, 4)


,Hair,Eye,Sex,Freq
1,Black,Brown,Male,32
2,Brown,Brown,Male,53
3,Red,Brown,Male,10
4,Blond,Brown,Male,3
5,Black,Blue,Male,11


**Question:** 

What do you notice about the structure of this dataset?


_____________________________________________

**Question:** 

If we take a look at a single row, what/who does it represent? 

In [13]:
# write the code to view a single row


____________________________________________


Our data has been aggregated! UGH! I *strongly dislike* starting with aggregated data, as a data scientist :=| It limits what I can find out. 

I want my data in the form where one row represents one observation. 


**Question:**

In this scenario, based on the description of the dataset, a single observation should be what?  


___________________________________________


So, let's stretch this data out, so that each row represents a single student. 

To do so, we need to ... 

1. repeat each row by the frequency. 

2. remove the frequency column. 

We should end up the same number of rows as the total number of students represented, because...

EACH ROW IS AN OBSERVATION AND EACH OBSERVATION IS A STUDENT. 


**Question:**

So, how can I compute how many rows we will end up with? Or how many students were surveyed?

If you need to take another peek at the data, go for it! `df.head()` is a quick and easy way. 

**Answer:**

In [15]:
# write code to compute how many students are represented in this dataset. 

Cool, so we are looking to end up with that many rows

___________________________________________
_______________________________________

To get there, we will want to repeat each combination of `Hair`, `Eye`, & `Sex` the number of times represented in the `Freq` field. We can get there in the following way:  

1. Create a single column that concatenates Hair, Eye & Sex

2. For the first unique combination of Hair, Eye & Sex, i.e. the first row which is Black, Brown, and Male, create a list that repeats that combination 32 times, which is the `Freq` value. 

3. We want to do that for each row, so once we made it work for one, we will put it in a loop to work for all. 

_______________________________________


> 1. Create a single column that concatenates Hair, Eye & Sex

In [16]:
# concatenating using '+' with strings

df['traits'] = df.Hair + ", " + df.Eye + ", " + df.Sex

> 2. Create a list from the first row, which is Black, Brown, and Male, that repeats that combination 32 times. 

In [17]:
# take a look at the first row
df.head(1)

,Hair,Eye,Sex,Freq,traits
1,Black,Brown,Male,32,"Black, Brown, Male"


In [18]:
# extract what we want to repeat: the contents in the traits column
traits = df.traits.iloc[0]
traits

'Black, Brown, Male'

In [19]:
# extract how many times we want to repeat...the value in the Freq column
freq = df.Freq.iloc[0]
freq

32

In [20]:
# use np.repeat to repeat `trait` `Freq` times.
traits_by_freq = np.repeat(df.traits.iloc[0], df.Freq.iloc[0])
traits_by_freq

array(['Black, Brown, Male', 'Black, Brown, Male', 'Black, Brown, Male',
       'Black, Brown, Male', 'Black, Brown, Male', 'Black, Brown, Male',
       'Black, Brown, Male', 'Black, Brown, Male', 'Black, Brown, Male',
       'Black, Brown, Male', 'Black, Brown, Male', 'Black, Brown, Male',
       'Black, Brown, Male', 'Black, Brown, Male', 'Black, Brown, Male',
       'Black, Brown, Male', 'Black, Brown, Male', 'Black, Brown, Male',
       'Black, Brown, Male', 'Black, Brown, Male', 'Black, Brown, Male',
       'Black, Brown, Male', 'Black, Brown, Male', 'Black, Brown, Male',
       'Black, Brown, Male', 'Black, Brown, Male', 'Black, Brown, Male',
       'Black, Brown, Male', 'Black, Brown, Male', 'Black, Brown, Male',
       'Black, Brown, Male', 'Black, Brown, Male'], dtype='<U18')

In [21]:
# Finally, we need that array to be a list
row1_observations = list(traits_by_freq)
row1_observations[0:5]

['Black, Brown, Male',
 'Black, Brown, Male',
 'Black, Brown, Male',
 'Black, Brown, Male',
 'Black, Brown, Male']

In [30]:
# pulling that all together...

row1_obs = list(np.repeat(df.traits.iloc[0], df.Freq.iloc[0]))

print("First original row:\n", df.iloc[0])
print("\nNumber of observations/students:\n",len(row1_obs))
print("\nFirst 5 observations:\n", row1_obs[0:5])

First original row:
 Hair                   Black
Eye                    Brown
Sex                     Male
Freq                      32
traits    Black, Brown, Male
Name: 1, dtype: object

Number of rows:
 32

First 5 rows:
 ['Black, Brown, Male', 'Black, Brown, Male', 'Black, Brown, Male', 'Black, Brown, Male', 'Black, Brown, Male']


> 3. We want to do that for each row, so let's make sure it works for another row, concatenate them, and then write a loop to do that for all the rows. 

In [33]:
row2_obs = list(np.repeat(df.traits.iloc[1], df.Freq.iloc[1]))


print("Second original row:\n", df.iloc[1])
print("\nNumber of observations/students:\n",len(row2_obs))
print("\nFirst 5 observations:\n", row2_obs[0:5])

Second original row:
 Hair                   Brown
Eye                    Brown
Sex                     Male
Freq                      53
traits    Brown, Brown, Male
Name: 2, dtype: object

Number of observations/students:
 53

First 5 observations:
 ['Brown, Brown, Male', 'Brown, Brown, Male', 'Brown, Brown, Male', 'Brown, Brown, Male', 'Brown, Brown, Male']


In [35]:
# concatenate observations from the first original row to the observations from the second
# start with an empty list to hold all observations
obs = []
# add the first rows observations to the list
obs += row1_obs

len(obs)

32

In [36]:
# add the second row's observations to the list
obs += row2_obs

len(obs)

85

In [37]:
# write the loop 
obs = []

# go through all 32 rows
for i in range(len(df)):
    obs += list(np.repeat(df.traits.iloc[i], df.Freq.iloc[i]))

In [38]:
# create a dataframe from the list. 
obs_df = pd.DataFrame({'traits': obs})
obs_df.head(3)

,traits
0,"Black, Brown, Male"
1,"Black, Brown, Male"
2,"Black, Brown, Male"


In [39]:
# split the traits at the comma into 3 different columns (n=-1 means ALL, so I could put 3 here since I know there are 3)
# expand = True means return to me a dataframe
obs_df = obs_df["traits"].str.split(", ", n = -1, expand = True)
obs_df.head(3)

,0,1,2
0,Black,Brown,Male
1,Black,Brown,Male
2,Black,Brown,Male


In [40]:
# give columns meaningful names
obs_df.columns = ['hair', 'eye', 'sex']
obs_df.head(3)

,hair,eye,sex
0,Black,Brown,Male
1,Black,Brown,Male
2,Black,Brown,Male


Now we are ready to create a list from the second row and test out concatenating these. 

In [41]:
# repeat the 2nd row `traits` the 2nd row `Freq` number of times, and turn that into a list

row2_observations = list(np.repeat(df.traits.iloc[1], df.Freq.iloc[1]))
row2_observations[0:5]

['Brown, Brown, Male',
 'Brown, Brown, Male',
 'Brown, Brown, Male',
 'Brown, Brown, Male',
 'Brown, Brown, Male']

In [42]:
# create an empty list, observations, to hold those from row1 and row2 lists 
observations = []

# add row1_observations to the list
observations += row1_observations
observations[0:5]

['Black, Brown, Male',
 'Black, Brown, Male',
 'Black, Brown, Male',
 'Black, Brown, Male',
 'Black, Brown, Male']

In [43]:
# Add row2_observations to the list
observations += row2_observations
observations[0:5]

['Black, Brown, Male',
 'Black, Brown, Male',
 'Black, Brown, Male',
 'Black, Brown, Male',
 'Black, Brown, Male']

So, now we know:

1. start with an empty list:  `observations = []`

2. in the loop will be: `observations += list(np.repeat(df.traits.iloc[i], df.Freq.iloc[i]))`

3. **Question:** What values are we going to loop through? `for i in ....`

In [44]:
# loop here



___________________________________________________
__________________________________________________


2. Sample vs. Population


According to heffingtons, the estimated population proportion of blue eys in the U.S. is 27%
This proportion from our sample seems much higher than expected! 

https://heffingtons.com/interesting-facts-about-eye-color/

- Brown Eyes: 45%

- Blue Eyes: 27%

- Hazel Eyes: 18% (Note: Hazel eyes consist of shades of brown and green.)

- Green Eyes: 9%

- Other: 1%


**Question:**

What are some possible reasons why? 




This dataset claims to be a sample of statistics students. 

**Questions:**

So is the population all statistics students? 

Across the world? 

All ages? 

Is it a representative sample of the population of all statistics students across the world of all ages? 

Or is it representative sample of the population of all people across the world of all ages? 

Or is it representative sample of the population of all people in the US of all ages? 

Or is it representative sample of the population of all statistics students in the US of all ages? 

Or is it representative sample of the population of all statistics students in the US of all ages? 

Or ...


_________________________________________________________


Hypothesis testing can help us answer these questions as well as many others. 


## Ways to answer questions that we will practice throughout your time here:

1. data visualization 

2. hypothesis testing

3. machine learning

4. Saying I don't know, but I will try to find out. 


Ways that we will not cover:

1. pulling it out of your arse

2. using statistics to give you the answer you want. 

3. selecting biased samples to prove your point. 

